In [1]:
import ipyvolume as ipv
import numpy as np

from compas_vol.primitives import VolBox, VolPlane, VolCylinder
from compas_vol.microstructures import TPMS
from compas_vol.combinations import Intersection, Union, Subtraction
from compas_vol.modifications import Overlay, Shell

from compas.geometry import Box, Frame, Point, Plane, Cylinder, Circle

In [2]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
def display(dm):
    fig = ipv.figure()
    ipv_mesh = ipv.plot_isosurface(dm, 0.0, color = 'white', extent = [[-130,145], [-16, 16], [-8,8]] )
    ipv.xyzlim(145)
    ipv.show()
    return ipv_mesh

In [4]:
x,y,z = np.ogrid[-130:145:550j, -16:16:64j, -8:8:32j]

In [5]:
vbox = VolBox(Box(Frame.worldXY(), 250, 30, 10), 1.5)
cyl = VolCylinder(Cylinder(Circle(Plane((125,0,0), (0,0,1)), 15), 10))
box_cylinder_union = Union(vbox, cyl)

dm = box_cylinder_union.get_distance_numpy(x,y,z)
display(dm)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [10]:
gyroid = TPMS(tpmstype = 'Gyroid', wavelength = 5.0)
shell = Shell(gyroid, 2.0, 0.5)
vplane = VolPlane(Plane((0,0,0), (1,0,0)))
overlay1 = Overlay(shell, vplane, 0.005)
intersection = Intersection(overlay1, box_cylinder_union)
c1 = VolCylinder(Cylinder(Circle(Plane((125,0,0),(0,0,1)), 12), 13))
c2 = VolCylinder(Cylinder(Circle(Plane((125,0,0),(0,0,1)), 10), 20))
add = Union(intersection, c1)
hole = Subtraction(add, c2)

In [11]:
dm = hole.get_distance_numpy(x,y,z)
final_ipv_mesh = display(dm)

In [8]:
def export_ipv_mesh(mesh, filename='ipv_mesh.obj', colors=None):
    """
    Exports a mesh of type :class:`ipyvolume.widgets.Mesh`
    to an .obj file at specified location.
    """
    import numpy as np
    import os
    vs = np.vstack((mesh.x, mesh.y, mesh.z)).T
    # vs = ['v {:.4f} {:.4f} {:.4f}'.format(v[0], v[1], v[2]) for v in vs]
    vs = ['v {} {} {}'.format(v[0], v[1], v[2]) for v in vs]
    if colors is not None:
        vs = [vs[i]+' {} {} {}'.format(c[0], c[1], c[2]) for i, c in enumerate(colors)]
    fs = ['f {} {} {}'.format(v[0]+1, v[1]+1, v[2]+1) for v in mesh.triangles]
    
    filename = os.path.join(os.getcwd(), filename)
    with open(filename, 'w') as f:
        f.write('\n'.join(vs))
        f.write('\n')
        f.write('\n'.join(fs))
    print("Export completed at " + str(filename))

In [9]:
export_ipv_mesh(final_ipv_mesh, 'overlays.obj')

Export completed at I:\Desktop\tutorial_28_oct\overlays.obj
